In [23]:
%load_ext autoreload
%autoreload 2

import os
curpath = os.getcwd()
os.chdir(curpath.split("core")[0])

In [28]:
import openai
import json
import time
from typing import List, Dict, Any, Tuple, Union
import urllib3
from xml.dom.minidom import parseString
from core.features.provider import creator, creator_defaults

# from prompts import SIMPLE_TS_PROMPTS, SIMPLE_TS_WITH_REF_PROMPTS
# from core.features.topic_segregation.utils import add_dicts, calculate_cost_gpt4_8k

from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
creator(**creator_defaults, messages=[{'role': 'user', 'content': 'Hello, world!'}])

In [27]:
def get_google_news_results(term:str, count: int):
    http = urllib3.PoolManager()
    results = []
    response = http.request('GET', 'http://news.google.com/news?q=%s&lang=en&output=rss' % term)
    obj = parseString(response.data.decode('utf-8'))
    items = obj.getElementsByTagName('item')  # Get each item
    for item in items[:count]:
        title, link = '', ''
        for node in item.childNodes:
            if node.nodeName == 'title':
                title = node.childNodes[0].data
            elif node.nodeName == 'link':
                link = node.childNodes[0].data
        results.append((title, link))
    return results

In [31]:
results = get_google_news_results('Rajasthan%20latest%20news', 100)
titles = []
links = []
for title, link in results:
    titles.append(title)
    links.append(link)

In [24]:
links[0]

'https://news.google.com/rss/articles/CBMiSmh0dHBzOi8vdGhld2lyZS5pbi9wb2xpdGljcy9yYWphc3RoYW4tY29uZ3Jlc3MtbWFkaHlhLXByYWRlc2gtY2hoYXR0aXNnYXJo0gFYaHR0cHM6Ly9tLnRoZXdpcmUuaW4vYXJ0aWNsZS9wb2xpdGljcy9yYWphc3RoYW4tY29uZ3Jlc3MtbWFkaHlhLXByYWRlc2gtY2hoYXR0aXNnYXJoL2FtcA?oc=5'

In [25]:
titles[0]

'Win or Lose: Rajasthan Shows The Way Forward for the Congress - The Wire'

In [32]:
title_txt = "\n".join(titles)

with open ('news_data_raj_en.txt', "wb") as f:
    f.write(title_txt.encode('utf-8'))

In [13]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import time

class InformationScraper:
    def __init__(self, queries, frequency='daily'):
        self.queries = queries
        self.frequency = frequency
        self.results = []

    def fetch_news(self, query):
        # Replace this URL with the actual news site or API you're planning to scrape
        url = f'https://news.google.com/search?q={query}'
        response = requests.get(url)
        if response.status_code == 200:
            return response.text
        else:
            print(f"Failed to retrieve news for query: {query}")
            return None

    def parse_news(self, html_content):
        soup = BeautifulSoup(html_content, 'html.parser')
        # Modify the selectors based on the actual structure of the news site
        articles = soup.find_all('article')
        news_items = []
        for article in articles:
            title = article.find('h3').get_text() if article.find('h3') else 'No Title'
            link = article.find('a', href=True)['href'] if article.find('a', href=True) else 'No Link'
            news_items.append({'title': title, 'link': link})
        return news_items

    def scrape(self):
        for query in self.queries:
            html_content = self.fetch_news(query)
            if html_content:
                news_items = self.parse_news(html_content)
                self.results.extend(news_items)

    def run(self):
        if self.frequency == 'daily':
            while True:
                print(f"Starting scrape at {datetime.now()}")
                self.scrape()
                print(f"Scrape complete at {datetime.now()}")
                break
                time.sleep(86400)  # Sleep for 1 day
        elif self.frequency == 'hourly':
            while True:
                print(f"Starting scrape at {datetime.now()}")
                self.scrape()
                print(f"Scrape complete at {datetime.now()}")
                break
                time.sleep(3600)  # Sleep for 1 hour
        # Add more frequency options as needed

    def get_results(self):
        return self.results

# Example usage:
queries = ['india hindi news']
scraper = InformationScraper(queries, frequency='daily')
scraper.run()


Starting scrape at 2023-11-10 23:44:20.664216
Scrape complete at 2023-11-10 23:44:22.731896


In [14]:
links = []
gnews = "https://news.google.com"

for res in scraper.results:
    links.append(gnews + res['link'][1:])


In [15]:
len(links)
links[:4]

['https://news.google.com/articles/CBMib2h0dHBzOi8vd3d3LmxpdmVoaW5kdXN0YW4uY29tL25hdGlvbmFsL3N0b3J5LWluZGlhLXVzLWNvLXByb2R1Y2UtYXJtb3JlZC12ZWhpY2xlcy10by1jb3VudGVyLWNoaW5hLTg5NjI5MjIuaHRtbNIBc2h0dHBzOi8vd3d3LmxpdmVoaW5kdXN0YW4uY29tL25hdGlvbmFsL3N0b3J5LWluZGlhLXVzLWNvLXByb2R1Y2UtYXJtb3JlZC12ZWhpY2xlcy10by1jb3VudGVyLWNoaW5hLTg5NjI5MjIuYW1wLmh0bWw?hl=en-IN&gl=IN&ceid=IN%3Aen',
 'https://news.google.com/articles/CBMicWh0dHBzOi8vd3d3LmFtYXJ1amFsYS5jb20vbGl2ZS9pbmRpYS1uZXdzL2xhdGVzdC1hbmQtYnJlYWtpbmctbmV3cy10b2RheS1pbi1oaW5kaS1saXZlLTktbm92ZW1iZXItMjAyMy0yMDIzLTExLTA50gF1aHR0cHM6Ly93d3cuYW1hcnVqYWxhLmNvbS9hbXAvbGl2ZS9pbmRpYS1uZXdzL2xhdGVzdC1hbmQtYnJlYWtpbmctbmV3cy10b2RheS1pbi1oaW5kaS1saXZlLTktbm92ZW1iZXItMjAyMy0yMDIzLTExLTA5?hl=en-IN&gl=IN&ceid=IN%3Aen',
 'https://news.google.com/articles/CBMipAFodHRwczovL3d3dy5hYnBsaXZlLmNvbS9uZXdzL3RvZGF5LXMtdG9wLTEwLW5ld3MtaGVhZGxpbmVzLXRvZGF5LWFicC1uZXdzLW1vcm5pbmctaGVhZGxpbmVzLTEwLW5vdmVtYmVyLTIwMjMtdG9wLW5ld3MtaGVhZGxpbmVzLXVwZGF0ZXMtZnJvbS1pbmRpYS1hbm

In [17]:
import requests
from bs4 import BeautifulSoup, TemplateString
from tqdm import tqdm
import time

def scraper(urls):
    
    results = []
    for url in tqdm(urls):

        # Send a GET request to the URL
        try:
            response = requests.get(url)
            time.sleep(0.5)
        except:
            print(f'Failed to get {url}')
            continue

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the content of the page using BeautifulSoup
            soup = BeautifulSoup(response.content, 'html.parser')
        
            for data in soup(['head', 'header', 'style', 'script', 'g', 'polygon', 'path', 'svg', 'option', 'footer']):
                # Remove tags
                data.decompose()

            article_text = ' '.join(paragraph.text.strip() for paragraph in soup.find_all('p'))
            
            # Extract the headline
            headline = soup.find('h1')
            if headline:
                headline = headline.text.strip()
            else:
                headline = ''

            result = headline + '\n' + article_text
            results.append(result)

        else:
            print("Failed to retrieve the webpage. Status code:", response.status_code)
            print(f"Error in URL: {url}")

    return results


In [18]:
results = scraper(links[:50])

  2%|▏         | 1/50 [00:01<01:21,  1.66s/it]

Failed to retrieve the webpage. Status code: 403
Error in URL: https://news.google.com/articles/CBMib2h0dHBzOi8vd3d3LmxpdmVoaW5kdXN0YW4uY29tL25hdGlvbmFsL3N0b3J5LWluZGlhLXVzLWNvLXByb2R1Y2UtYXJtb3JlZC12ZWhpY2xlcy10by1jb3VudGVyLWNoaW5hLTg5NjI5MjIuaHRtbNIBc2h0dHBzOi8vd3d3LmxpdmVoaW5kdXN0YW4uY29tL25hdGlvbmFsL3N0b3J5LWluZGlhLXVzLWNvLXByb2R1Y2UtYXJtb3JlZC12ZWhpY2xlcy10by1jb3VudGVyLWNoaW5hLTg5NjI5MjIuYW1wLmh0bWw?hl=en-IN&gl=IN&ceid=IN%3Aen


 58%|█████▊    | 29/50 [02:57<06:10, 17.66s/it]

Failed to get https://news.google.com/articles/CBMikQFodHRwczovL3d3dy5hYWp0YWsuaW4vaW5kaWEvbmV3cy9zdG9yeS9hYWota2ktdGF6YS1raGFiYXItMTQtb2N0b2Jlci0yMDIzLWxhdGVzdC10YWFqYS1zYW1hY2hhci1saXZlLW5ld3MtdXBkYXRlcy1pbi1oaW5kaS1udGMtMTc5OTAxMi0yMDIzLTEwLTE00gGVAWh0dHBzOi8vd3d3LmFhanRhay5pbi9hbXAvaW5kaWEvbmV3cy9zdG9yeS9hYWota2ktdGF6YS1raGFiYXItMTQtb2N0b2Jlci0yMDIzLWxhdGVzdC10YWFqYS1zYW1hY2hhci1saXZlLW5ld3MtdXBkYXRlcy1pbi1oaW5kaS1udGMtMTc5OTAxMi0yMDIzLTEwLTE0?hl=en-IN&gl=IN&ceid=IN%3Aen


 66%|██████▌   | 33/50 [03:04<01:34,  5.58s/it]

...

 92%|█████████▏| 46/50 [04:13<00:15,  3.76s/it]

Failed to retrieve the webpage. Status code: 403
Error in URL: https://news.google.com/articles/CBMid2h0dHBzOi8vd3d3LmxpdmVoaW5kdXN0YW4uY29tL25hdGlvbmFsL3N0b3J5LWVsdmlzaC15YWRhdi1uZXdzLXNuYWtlLXBvaXNvbi1hZGRpY3Rpb24tYWxjb2hvbC1saXF1b3ItbmFzaGEtODk0NzQ2MC5odG1s0gF7aHR0cHM6Ly93d3cubGl2ZWhpbmR1c3Rhbi5jb20vbmF0aW9uYWwvc3RvcnktZWx2aXNoLXlhZGF2LW5ld3Mtc25ha2UtcG9pc29uLWFkZGljdGlvbi1hbGNvaG9sLWxpcXVvci1uYXNoYS04OTQ3NDYwLmFtcC5odG1s?hl=en-IN&gl=IN&ceid=IN%3Aen


 96%|█████████▌| 48/50 [04:16<00:05,  2.61s/it]

Failed to retrieve the webpage. Status code: 403
Error in URL: https://news.google.com/articles/CBMicGh0dHBzOi8vd3d3LmxpdmVoaW5kdXN0YW4uY29tL25hdGlvbmFsL3N0b3J5LW11bWJhaS1haXItcG9sbHV0aW9uLW11bWJhaS1jaG9raW5nLWZlZWxpbmctbGlrZS1kZWxoaS04OTUxNDg4Lmh0bWzSAXRodHRwczovL3d3dy5saXZlaGluZHVzdGFuLmNvbS9uYXRpb25hbC9zdG9yeS1tdW1iYWktYWlyLXBvbGx1dGlvbi1tdW1iYWktY2hva2luZy1mZWVsaW5nLWxpa2UtZGVsaGktODk1MTQ4OC5hbXAuaHRtbA?hl=en-IN&gl=IN&ceid=IN%3Aen


100%|██████████| 50/50 [04:19<00:00,  5.20s/it]


In [12]:
len(results)

with open("test_news_hindi.txt", 'wb') as f:
    for result in results:
        f.write(result.encode('utf-8').strip())

## trash

In [36]:
import requests
from bs4 import BeautifulSoup, TemplateString

# def scraper(url):
    # The URL of the article to scrape
url = "https://me.mashable.com/culture/34493/northern-lights-paint-bulgarias-sky-in-eerie-red-hue-see-pics"
url = "https://www.ndtv.com/india-news/cbi-inquiry-against-mahua-moitra-ordered-by-anti-corruption-panel-bjp-mp-4557198"
url = "https://thewire.in/security/india-armed-forces-politicisation-by-default-or-design"
url = "https://www.washingtonpost.com/politics/2023/11/07/abortion-ohio-kentucky-virginia-elecitons/"
url = "https://www.theinformation.com/articles/american-ai-startups-raise-money-from-top-chinese-vc-firms-including-sequoia-capital-china"
url = "https://www.nytimes.com/2023/11/08/business/energy-environment/electric-vehicles-biden.html"

    # Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the content of the page using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # soup = BeautifulSoup(html, "html.parser")
 
    for data in soup(['head', 'header', 'style', 'script', 'g', 'polygon', 'path', 'svg', 'option', 'footer']):
        # Remove tags
        data.decompose()
    # temp = TemplateString(response.content)
    print(soup)
    print()

    article_text = ' '.join(paragraph.text.strip() for paragraph in soup.find_all('p'))

    # print(article_text)
    # print(len(soup))
    # print(temp)
    # print(soup)

    # soup.find_all('script')
    # soup.find_all()
    
    # Extract the headline
    headline = soup.find('h1')
    if headline:
        headline = headline.text.strip()
    
#     # Try to find the container of the article content
#     # The class or tag here is hypothetical and should be replaced with the actual class or tag that contains the article text.
#     article_content = soup.find('article') 
#     # or soup.find('main')  # Adjust the class or tag as needed
    
#     # Check if the article content was found
    # if article_content:
    #     paragraphs = article_content.find_all('p')
    #     article_text = ' '.join(paragraph.text.strip() for paragraph in paragraphs)
    # else:
    #     article_text = "Could not find the article content."
    
    # Print the extracted information
    print("Headline:", headline)
    print("Article Text:", article_text)
# else:
#     print("Failed to retrieve the webpage. Status code:", response.status_code)

        
        


In [10]:
url = "https://www.ndtv.com/india-news/cbi-inquiry-against-mahua-moitra-ordered-by-anti-corruption-panel-bjp-mp-4557198"

scraper(url)

Headline: CBI Probe Against Mahua Moitra Ordered By Anti-Graft Panel, Claims BJP MP
Article Text: Anti-corruption body Lokpal has ordered a probe by the Central Bureau of Investigation into the corruption allegations against Trinamool Congress MP Mahua Moitra, the BJP's Nishikant Dubey posted today on X, formerly Twitter. Ms Moitra has been embroiled in cash for query allegations levelled by Mr Dubey, which is under the lens of the Parliamentary Ethics Committee. In a letter to Lok Sabha Speaker Om Birla, he had demanded her immediate suspension from parliament. "On basis of my complaint, Lokpal has ordered a CBI probe into the Mahua Moitra's corruption that compromises national security," read a rough translation of BJP MP's post. Nishikant Dubey has alleged that Ms Moitra accepted cash from businessman Darshan Hiranandani to ask questions in parliament on his behalf to target Prime Minister Narendra Modi and business rival Adani Group.  He has also claimed that Ms Moitra, by sharing 